In [1]:

import torch
import torchvision
print("PyTorch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)
print("CUDA is available:", torch.cuda.is_available())


PyTorch version: 2.5.1+cpu
Torchvision version: 0.20.1+cpu
CUDA is available: False


In [2]:
# =============================================================================
# 📦 IMPORTS ET CONFIGURATION ENVIRONNEMENT
# =============================================================================

import os
import json
import numpy as np
import torch
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import uuid
from datetime import datetime
from typing import List, Tuple, Dict, Any, Optional
from dataclasses import dataclass
from pathlib import Path
import base64
from pycocotools.mask import encode as encode_rle

# Configuration device
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

print(f"🖥️ Device utilisé: {device}")

# Optimisations CUDA
if device.type == "cuda":
    torch.autocast("cuda", dtype=torch.bfloat16).__enter__()
    if torch.cuda.get_device_properties(0).major >= 8:
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.allow_tf32 = True
        print("✅ Optimisations CUDA activées")

print("✅ Environnement configuré")


🖥️ Device utilisé: cpu
✅ Environnement configuré


In [3]:
# =============================================================================
# ⚙️ CONFIGURATION DU PROJET
# =============================================================================

# 📋 CONFIGURATION PRINCIPALE - MODIFIEZ CES VALEURS
VIDEO_NAME = "SD_13_06_2025_1_PdB_S1_T959s"  # ⚠️ Nom de la vidéo (sans extension)
VIDEOS_DIR = "../data/videos"   # 📁 Dossier contenant les vidéo
FRAME_INTERVAL = 3       # 🎬 Intervalle entre frames (1=toutes, 10=1 sur 10)

# 🎬 OPTIONS D'EXTRACTION DES FRAMES
EXTRACT_FRAMES = True     # ✅ True=Extraire, False=Skip extraction
FORCE_EXTRACTION = False  # 🔄 True=Forcer même si frames existent, False=Skip si existent

# 🗂️ Construction des chemins automatiques
video_path = Path(VIDEOS_DIR) / f"{VIDEO_NAME}.mp4"
config_path = Path(VIDEOS_DIR) / f"{VIDEO_NAME}_config.json"
output_dir = Path(VIDEOS_DIR) / "outputs" / VIDEO_NAME
frames_dir = output_dir / "frames"
masks_dir = output_dir / "masks"
output_video_path = output_dir / f"{VIDEO_NAME}_annotated.mp4"
output_json_path = output_dir / f"{VIDEO_NAME}_project.json"

# 🏗️ Création des dossiers
output_dir.mkdir(parents=True, exist_ok=True)
frames_dir.mkdir(exist_ok=True)
masks_dir.mkdir(exist_ok=True)

# 🔍 Vérification des fichiers
print(f"📋 CONFIGURATION DU PROJET:")
print(f"   🎬 Vidéo: {video_path}")
print(f"   📄 Config: {config_path}")
print(f"   📁 Sortie: {output_dir}")
print(f"   ⏯️  Intervalle frames: {FRAME_INTERVAL}")
print(f"   🎬 Extraction: {'✅ Activée' if EXTRACT_FRAMES else '❌ Désactivée'}")
print(f"   🔄 Force extraction: {'✅ Oui' if FORCE_EXTRACTION else '❌ Non'}")

# Vérifications
if not video_path.exists():
    raise FileNotFoundError(f"❌ Vidéo non trouvée: {video_path}")
if not config_path.exists():
    raise FileNotFoundError(f"❌ Fichier config non trouvé: {config_path}")

print("✅ Configuration validée")


📋 CONFIGURATION DU PROJET:
   🎬 Vidéo: ..\data\videos\SD_13_06_2025_1_PdB_S1_T959s.mp4
   📄 Config: ..\data\videos\SD_13_06_2025_1_PdB_S1_T959s_config.json
   📁 Sortie: ..\data\videos\outputs\SD_13_06_2025_1_PdB_S1_T959s
   ⏯️  Intervalle frames: 3
   🎬 Extraction: ✅ Activée
   🔄 Force extraction: ❌ Non
✅ Configuration validée


In [4]:
# =============================================================================
# 📄 CHARGEMENT DE LA CONFIGURATION JSON
# =============================================================================

def load_config_file(config_path: Path) -> Dict[str, Any]:
    """Charge et valide le fichier de configuration JSON."""
    print(f"📄 Chargement de la configuration: {config_path}")

    with open(config_path, 'r', encoding='utf-8') as f:
        config = json.load(f)

    # Validation de la structure
    required_sections = ['calibration', 'objects', 'initial_annotations']
    for section in required_sections:
        if section not in config:
            raise ValueError(f"❌ Section '{section}' manquante dans le config")

    # Statistiques
    num_objects = len(config['objects'])
    num_annotations = 0
    for frame_data in config['initial_annotations']:
        num_annotations += len(frame_data['annotations'])

    print(f"✅ Configuration chargée:")
    print(f"   📷 Calibration caméra: OK")
    print(f"   🎯 Objets définis: {num_objects}")
    print(f"   📍 Annotations initiales: {num_annotations}")

    # Résumé des types d'objets
    obj_types = {}
    for obj in config['objects']:
        obj_type = obj['obj_type']
        obj_types[obj_type] = obj_types.get(obj_type, 0) + 1
    print(f"   🏷️  Types: {dict(obj_types)}")

    return config

# Chargement de la configuration
config = load_config_file(config_path)


📄 Chargement de la configuration: ..\data\videos\SD_13_06_2025_1_PdB_S1_T959s_config.json
✅ Configuration chargée:
   📷 Calibration caméra: OK
   🎯 Objets définis: 11
   📍 Annotations initiales: 11
   🏷️  Types: {'ball': 1, 'player': 10}


In [5]:
config

{'calibration': {'status': 'success',
  'camera_parameters': {'mode': 'full',
   'use_ransac': 5,
   'rep_err': 5.677561117314194,
   'cam_params': {'pan_degrees': 42.19814348024774,
    'tilt_degrees': 63.36640628466144,
    'roll_degrees': 1.0557592747483122,
    'x_focal_length': 1567.6990141649394,
    'y_focal_length': 1567.6990141649394,
    'principal_point': [959.5, 539.5],
    'position_meters': [-62.3440986793476,
     32.78263355828649,
     -14.987520424641314],
    'rotation_matrix': [[0.735152508193547,
      0.677701643528148,
      0.016470338672408935],
     [-0.3147092720910286, 0.3196673869020791, 0.8937398031928149],
     [0.6004239033854928, -0.6622184262842915, 0.4482832721983776]],
    'radial_distortion': [0, 0, 0, 0, 0, 0],
    'tangential_distortion': [0, 0],
    'thin_prism_distortion': [0, 0, 0, 0]},
   'calib_plane': 0},
  'input_lines': {'Side line top': [{'x': 2.640748255461165,
     'y': 222.55601554081358},
    {'x': 971.9611366049758, 'y': 48.762912092

In [6]:
# =============================================================================
# 🎬 EXTRACTION DES FRAMES
# =============================================================================

def extract_frames(video_path: Path, frames_dir: Path, frame_interval: int = 1, force_extraction: bool = False) -> int:
    """Extrait les frames de la vidéo selon l'intervalle spécifié."""

    print(f"🎬 Extraction des frames...")
    print(f"   📹 Source: {video_path}")
    print(f"   📁 Destination: {frames_dir}")
    print(f"   ⏯️  Intervalle: {frame_interval}")
    print(f"   🔄 Force extraction: {'✅ Oui' if force_extraction else '❌ Non'}")

    # Vérification si extraction déjà faite
    existing_frames = list(frames_dir.glob("*.jpg"))
    if existing_frames and not force_extraction:
        print(f"📂 {len(existing_frames)} frames déjà extraites - SKIP")
        return len(existing_frames)
    elif existing_frames and force_extraction:
        print(f"🔄 {len(existing_frames)} frames existantes - SUPPRESSION et ré-extraction...")
        # Supprimer les frames existantes
        for frame_file in existing_frames:
            frame_file.unlink()
        print(f"🗑️  Frames existantes supprimées")

    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        raise ValueError(f"❌ Impossible d'ouvrir la vidéo: {video_path}")

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    print(f"📊 Vidéo: {total_frames} frames, {fps:.1f} FPS")

    extracted_count = 0
    frame_idx = 0

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            # Extraire seulement selon l'intervalle
            if frame_idx % frame_interval == 0:
                output_idx = frame_idx // frame_interval
                filename = frames_dir / f"{output_idx:05d}.jpg"
                cv2.imwrite(str(filename), frame, [cv2.IMWRITE_JPEG_QUALITY, 95])
                extracted_count += 1

                if extracted_count % 50 == 0:
                    progress = (frame_idx / total_frames) * 100
                    print(f"📊 Progrès: {extracted_count} frames extraites ({progress:.1f}%)")

            frame_idx += 1

    finally:
        cap.release()

    print(f"✅ {extracted_count} frames extraites")
    return extracted_count

# Extraction des frames
if EXTRACT_FRAMES:
    extracted_frames_count = extract_frames(video_path, frames_dir, FRAME_INTERVAL, FORCE_EXTRACTION)
else:
    # Skip extraction - compter les frames existantes
    existing_frames = list(frames_dir.glob("*.jpg"))
    extracted_frames_count = len(existing_frames)

    print(f"⏭️  Extraction désactivée")
    if extracted_frames_count > 0:
        print(f"📂 Utilisation de {extracted_frames_count} frames existantes")
    else:
        print(f"⚠️  Aucune frame trouvée dans {frames_dir}")
        print(f"💡 Conseil: Activez EXTRACT_FRAMES=True pour extraire les frames")


🎬 Extraction des frames...
   📹 Source: ..\data\videos\SD_13_06_2025_1_PdB_S1_T959s.mp4
   📁 Destination: ..\data\videos\outputs\SD_13_06_2025_1_PdB_S1_T959s\frames
   ⏯️  Intervalle: 3
   🔄 Force extraction: ❌ Non
📂 208 frames déjà extraites - SKIP


In [ ]:
# =============================================================================
# 🤖 INITIALISATION SAM2
# =============================================================================

# Import SAM2
from sam2.build_sam import build_sam2_video_predictor

# Configuration des chemins SAM2
checkpoint_path = "../checkpoints/sam2.1_hiera_large.pt"
model_config_path = "configs/sam2.1/sam2.1_hiera_l.yaml"

print(f"🤖 Initialisation SAM2...")
print(f"   🧠 Modèle: {model_config_path}")
print(f"   💾 Checkpoint: {checkpoint_path}")
print(f"   🖥️  Device: {device}")

# Construction du predictor
predictor = build_sam2_video_predictor(
    config_file=model_config_path,
    ckpt_path=checkpoint_path,
    device=device
)

# Initialisation de l'état d'inférence
print(f"\n🎬 Initialisation état d'inférence...")
print(f"   📁 Frames: {frames_dir}")

inference_state = predictor.init_state(
    video_path=str(frames_dir),
    offload_video_to_cpu=True,    # Économise la mémoire GPU
    offload_state_to_cpu=False    # Garde l'état en GPU
)

# Reset de l'état
predictor.reset_state(inference_state)

# Vérification
loaded_frames = inference_state["num_frames"]
print(f"\n✅ SAM2 initialisé:")
print(f"   🖼️  Frames extraites: {extracted_frames_count}")
print(f"   🎬 Frames chargées: {loaded_frames}")
print(f"   ✅ Correspondance: {'OK' if extracted_frames_count == loaded_frames else 'ERREUR'}")

if device.type == "cuda":
    allocated = torch.cuda.memory_allocated() / 1024**3
    print(f"   💾 GPU Memory: {allocated:.2f}GB")


🤖 Initialisation SAM2...
   🧠 Modèle: configs/sam2.1/sam2.1_hiera_l.yaml
   💾 Checkpoint: ../checkpoints/sam2.1_hiera_large.pt
   🖥️  Device: cpu

🎬 Initialisation état d'inférence...
   📁 Frames: ..\data\videos\outputs\SD_13_06_2025_1_PdB_S1_T959s\frames


frame loading (JPEG): 100%|██████████| 208/208 [00:17<00:00, 11.57it/s]


In [ ]:
# =============================================================================
# 🎯 AJOUT DES ANNOTATIONS INITIALES
# =============================================================================

def add_initial_annotations(predictor, inference_state, config: Dict[str, Any]):
    """Ajoute les annotations initiales depuis la configuration."""

    print(f"🎯 Ajout des annotations initiales...")

    # Création du mapping obj_id -> obj_type
    obj_types = {}
    for obj in config['objects']:
        obj_types[obj['obj_id']] = obj['obj_type']

    # Extraction automatique des annotations et frames depuis le JSON
    all_annotations = []
    annotation_frames = []

    for frame_data in config['initial_annotations']:
        frame_idx = frame_data['frame']
        annotations = frame_data['annotations']
        annotation_frames.append(frame_idx)

        print(f"   📍 Frame {frame_idx}: {len(annotations)} annotations")

        for annotation in annotations:
            all_annotations.append({
                'frame': frame_idx,
                'obj_id': annotation['obj_id'],
                'points': annotation['points'],
                'obj_type': obj_types.get(annotation['obj_id'], f'unknown_{annotation["obj_id"]}')
            })

    if not all_annotations:
        raise ValueError(f"❌ Aucune annotation trouvée dans le fichier config")

    print(f"   📊 Total: {len(all_annotations)} annotations sur {len(set(annotation_frames))} frames")

    # Ajout des annotations à SAM2
    added_objects = []

    for annotation_data in all_annotations:
        frame_idx = annotation_data['frame']
        obj_id = annotation_data['obj_id']
        obj_type = annotation_data['obj_type']
        points_data = annotation_data['points']

        # Extraction des coordonnées et labels
        points = np.array([[p['x'], p['y']] for p in points_data], dtype=np.float32)
        labels = np.array([p['label'] for p in points_data], dtype=np.int32)

        print(f"   🎯 Frame {frame_idx} - Objet {obj_id} ({obj_type}): {len(points)} points à ({points[0][0]:.0f}, {points[0][1]:.0f})")

        # Ajout à SAM2 avec add_new_points_or_box
        _, out_obj_ids, out_mask_logits = predictor.add_new_points_or_box(
            inference_state,
            frame_idx,
            obj_id,
            points,
            labels
        )

        # Éviter les doublons dans added_objects
        if not any(obj['obj_id'] == obj_id for obj in added_objects):
            added_objects.append({
                'obj_id': obj_id,
                'obj_type': obj_type,
                'points_count': len(points)
            })

    # Vérification
    sam_obj_ids = inference_state["obj_ids"]

    print(f"\n📊 RÉSUMÉ ANNOTATIONS:")
    print(f"   🎯 Annotations configurées: {len(all_annotations)}")
    print(f"   🎯 Objets uniques: {len(added_objects)}")
    print(f"   ✅ Objets ajoutés à SAM2: {len(sam_obj_ids)}")
    print(f"   🆔 IDs: {sorted(sam_obj_ids)}")
    print(f"   📍 Frames utilisées: {sorted(set(annotation_frames))}")

    # Résumé par type
    type_counts = {}
    for obj in added_objects:
        obj_type = obj['obj_type']
        type_counts[obj_type] = type_counts.get(obj_type, 0) + 1
    print(f"   🏷️  Types: {dict(type_counts)}")

    return added_objects, all_annotations

# Ajout des annotations
added_objects, initial_annotations_data = add_initial_annotations(predictor, inference_state, config)
print("\n✅ Annotations initiales ajoutées avec succès!")


In [9]:
# config['calibration']['camera_parameters']

In [ ]:
# =============================================================================
# 🔄 PROPAGATION ET GÉNÉRATION DES ANNOTATIONS
# =============================================================================

def generate_frame_mapping(total_frames: int, frame_interval: int) -> List[Optional[int]]:
    """
    Génère le mapping entre frames originales et frames traitées.

    Args:
        total_frames: Nombre total de frames dans la vidéo originale
        frame_interval: Intervalle entre frames (ex: 10 = 1 frame sur 10)

    Returns:
        Liste où l'index = frame originale, valeur = frame traitée (ou None si pas traitée)
    """
    frame_mapping = [None] * total_frames
    processed_idx = 0

    for original_idx in range(total_frames):
        if original_idx % frame_interval == 0:
            frame_mapping[original_idx] = processed_idx
            processed_idx += 1

    return frame_mapping

def create_project_structure(config: Dict[str, Any], video_info: Dict[str, Any], added_objects: List[Dict]) -> Dict[str, Any]:
    """Crée la structure JSON du projet."""

    # Calcul du frame mapping
    frame_mapping = generate_frame_mapping(
        video_info['total_frames'],
        FRAME_INTERVAL
    )
    processed_frame_count = sum(x is not None for x in frame_mapping)

    # Structure objects avec couleurs
    import random
    import colorsys

    config_objects_mapping = {}
    for obj in config['objects']:
        config_objects_mapping[obj['obj_id']] = obj

    objects = {}
    for obj_data in added_objects:
        obj_id = str(obj_data['obj_id'])
        obj_type = obj_data['obj_type']

        # Récupérer les informations complètes depuis le config
        config_obj = config_objects_mapping.get(int(obj_id), {})

        # Couleur aléatoire reproductible
        random.seed(int(obj_id) * 12345)
        hue = random.random()
        rgb = colorsys.hsv_to_rgb(hue, 0.8, 0.9)
        hex_color = "#{:02x}{:02x}{:02x}".format(
            int(rgb[0] * 255), int(rgb[1] * 255), int(rgb[2] * 255)
        )

        objects[obj_id] = {
            "id": obj_id,
            "type": obj_type,
            "team": config_obj.get('team', None),  # ← Récupéré depuis le config
            "jersey_number": config_obj.get('jersey_number', None),
            "jersey_color": config_obj.get('jersey_color', None),
            "role": config_obj.get('role', None),
            "display_color": hex_color
        }

    return {
        "format_version": "1.0",
        "video": f"{VIDEO_NAME}.mp4",
        "metadata": {
            "project_id": str(uuid.uuid4()),
            "created_at": datetime.now().isoformat() + "Z",
            "fps": video_info['fps'],
            "resolution": {
                "width": video_info['width'],
                "height": video_info['height'],
                "aspect_ratio": round(video_info['width'] / video_info['height'], 2)
            },
            "frame_interval": FRAME_INTERVAL,
            "frame_count_original": video_info['total_frames'],
            "frame_count_processed": processed_frame_count,
            "frame_mapping": frame_mapping,
            "static_video": False
        },
        "calibration": config['calibration'],
        "objects": objects,
        "initial_annotations": config['initial_annotations'],  # ← Annotations initiales depuis config
        "annotations": {}
    }

# =============================================================================
# 🔧 FONCTIONS UTILITAIRES POUR ANNOTATIONS COMPLÈTES
# =============================================================================

def get_object_scores(predictor, inference_state, frame_idx, obj_id):
    """Récupère les scores d'objet de manière propre et sûre"""
    try:
        obj_idx = predictor._obj_id_to_idx(inference_state, obj_id)
        obj_output_dict = inference_state["output_dict_per_obj"][obj_idx]
        temp_output_dict = inference_state["temp_output_dict_per_obj"][obj_idx]

        # Chercher dans les outputs
        frame_output = None
        if frame_idx in temp_output_dict["cond_frame_outputs"]:
            frame_output = temp_output_dict["cond_frame_outputs"][frame_idx]
        elif frame_idx in temp_output_dict["non_cond_frame_outputs"]:
            frame_output = temp_output_dict["non_cond_frame_outputs"][frame_idx]
        elif frame_idx in obj_output_dict["cond_frame_outputs"]:
            frame_output = obj_output_dict["cond_frame_outputs"][frame_idx]
        elif frame_idx in obj_output_dict["non_cond_frame_outputs"]:
            frame_output = obj_output_dict["non_cond_frame_outputs"][frame_idx]

        if frame_output and "object_score_logits" in frame_output:
            object_score_logits = frame_output["object_score_logits"]
            return torch.sigmoid(object_score_logits).item()
        return None

    except Exception as e:
        return None

def calculate_bbox_from_rle(rle_data: Dict[str, Any]) -> Optional[Dict[str, int]]:
    """Calcule la bounding box depuis un RLE base64."""
    from pycocotools.mask import toBbox

    try:
        rle = {
            "size": rle_data["size"],
            "counts": base64.b64decode(rle_data["counts"])
        }

        bbox = toBbox(rle)

        result = {
            "x": int(bbox[0]),
            "y": int(bbox[1]),
            "width": int(bbox[2]),
            "height": int(bbox[3])
        }
        return result

    except Exception as e:
        print(f"⚠️ Erreur calcul bbox: {e}")
        return None



def image_to_world(point_2d, cam_params):
    """
    Projette un point 2D de l'image vers le plan du terrain (Z=0).
    """
    # Create projection matrix P
    K = np.array([
        [cam_params["cam_params"]["x_focal_length"], 0, cam_params["cam_params"]["principal_point"][0]],
        [0, cam_params["cam_params"]["y_focal_length"], cam_params["cam_params"]["principal_point"][1]],
        [0, 0, 1]
    ])
    R = np.array(cam_params["cam_params"]["rotation_matrix"])
    t = -R @ np.array(cam_params["cam_params"]["position_meters"])
    P = K @ np.hstack((R, t.reshape(-1,1)))

    # Create point on image plane in homogeneous coordinates
    point_2d_h = np.array([point_2d[0], point_2d[1], 1])

    # Back-project ray from camera
    ray = np.linalg.inv(K) @ point_2d_h
    ray = R.T @ ray

    # Find intersection with Z=0 plane
    camera_pos = np.array(cam_params["cam_params"]["position_meters"])
    t = -camera_pos[2] / ray[2]
    world_point = camera_pos + t * ray

    return world_point[:2]  # Return only X,Y coordinates since Z=0

def calculate_points_output(bbox_output: dict, cam_params: dict = None) -> dict:
    """
    Calcule les points de sortie à partir de la bbox output.

    Args:
        bbox_output: Dict avec 'x', 'y', 'width', 'height'
        cam_params: Paramètres de calibration caméra pour projection terrain

    Returns:
        Dict avec les points calculés séparés par plan (image vs field)
    """
    if not bbox_output:
        return None

    # Calculer le point CENTER_BOTTOM dans le plan image
    center_bottom_x = bbox_output['x'] + bbox_output['width'] / 2
    center_bottom_y = bbox_output['y'] + bbox_output['height']  # Bas de la bbox

    # Structure avec séparation image/field
    points_output = {
        "image": {
            "CENTER_BOTTOM": {
                "x": float(center_bottom_x),
                "y": float(center_bottom_y)
            }
        },
        "field": {
            "CENTER_BOTTOM": None
        }
    }

    # Projection vers le terrain si les paramètres caméra sont fournis
    if cam_params:
        try:
            # Projeter le point CENTER_BOTTOM vers le terrain
            image_point = [center_bottom_x, center_bottom_y]
            field_point = image_to_world(image_point, cam_params)

            points_output["field"]["CENTER_BOTTOM"] = {
                "x": float(field_point[0]),
                "y": float(field_point[1])
            }

        except Exception as e:
            print(f"⚠️ Erreur projection terrain: {e}")
            points_output["field"]["CENTER_BOTTOM"] = None

    return points_output

def create_mask_annotation(obj_id: int, mask_logits, predictor=None, inference_state=None,
                         frame_idx=None, cam_params: Dict = None) -> Dict:
    """
    Crée une annotation de masque complète avec points input/output, bbox et scores.
    """
    # Conversion en masque binaire
    mask = (mask_logits > 0.0).cpu().numpy()
    if mask.ndim == 3 and mask.shape[0] == 1:
        mask = np.squeeze(mask, axis=0)

    # Encodage RLE
    if not mask.flags['F_CONTIGUOUS']:
        mask = np.asfortranarray(mask)

    rle = encode_rle(mask.astype(np.uint8))
    base64_counts = base64.b64encode(rle["counts"]).decode('ascii')

    # Calcul bbox et points output si masque non vide
    bbox_output = None
    points_output = None

    if mask.sum() > 0:
        from pycocotools.mask import toBbox
        bbox = toBbox(rle)
        bbox_output = {
            "x": int(bbox[0]),
            "y": int(bbox[1]),
            "width": int(bbox[2]),
            "height": int(bbox[3])
        }
        # Calcul des points output depuis la bbox
        points_output = calculate_points_output(bbox_output, cam_params)
    # else:
    #     print(f"⚠️ Masque vide pour l'objet {obj_id}, bbox et points output = None")

    # Récupération du score du masque
    mask_score = None

    if predictor and inference_state and frame_idx is not None:
        # Score du masque
        mask_score = get_object_scores(predictor, inference_state, frame_idx, obj_id)

    # Structure d'annotation complète
    return {
        "id": str(uuid.uuid4()),
        "objectId": str(obj_id),
        "type": "mask",
        "mask": {
            "format": "rle_coco_base64",
            "size": [int(rle["size"][0]), int(rle["size"][1])],
            "counts": base64_counts
        },
        "bbox": {
            "output": bbox_output
        },
        "points": {
            "output": points_output
        },
        "maskScore": mask_score,
        "pose": None,
        "warning": False
    }

# Informations vidéo
cap = cv2.VideoCapture(str(video_path))
video_info = {
    'total_frames': int(cap.get(cv2.CAP_PROP_FRAME_COUNT)),
    'fps': cap.get(cv2.CAP_PROP_FPS),
    'width': int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
    'height': int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
}
cap.release()

# Création du projet
project = create_project_structure(config, video_info, added_objects)

print(f"🔄 Démarrage de la propagation...")
print(f"   🎬 {extracted_frames_count} frames à traiter")
print(f"   🎯 {len(added_objects)} objets à suivre")

# Propagation et annotation
for out_frame_idx, out_obj_ids, out_mask_logits in predictor.propagate_in_video(inference_state):

    if str(out_frame_idx) not in project['annotations']:
        project['annotations'][str(out_frame_idx)] = []

    for i, out_obj_id in enumerate(out_obj_ids):
        annotation = create_mask_annotation(
            obj_id=out_obj_id,
            mask_logits=out_mask_logits[i],
            predictor=predictor,
            inference_state=inference_state,
            frame_idx=out_frame_idx,
            cam_params=config['calibration']['camera_parameters']
        )
        project['annotations'][str(out_frame_idx)].append(annotation)


In [ ]:
project['objects']

In [ ]:
# =============================================================================
# 💾 SAUVEGARDE DES RÉSULTATS
# =============================================================================

print(f"💾 Sauvegarde des résultats...")

# Sauvegarde du JSON
with open(output_json_path, 'w', encoding='utf-8') as f:
    json.dump(project, f, indent=2, ensure_ascii=False)

print(f"✅ Fichier JSON sauvé: {output_json_path}")

# Statistiques finales
total_annotations = sum(len(annotations) for annotations in project['annotations'].values())
unique_frames = len(project['annotations'])


print(f"\n📊 RÉSULTATS FINAUX:")
print(f"   🎬 Frames originales: {project['metadata']['frame_count_original']}")
print(f"   🎬 Frames traitées: {project['metadata']['frame_count_processed']}")
print(f"   📍 Frames avec annotations: {unique_frames}")
print(f"   📍 Annotations totales (propagées): {total_annotations}")
print(f"   🎯 Objets suivis: {len(project['objects'])}")
print(f"   ⏯️  Intervalle: {project['metadata']['frame_interval']}")
print(f"   📄 Fichier de sortie: {output_json_path}")
print(f"   📁 Dossier de sortie: {output_dir}")

# Affichage d'un échantillon du mapping
sample_mapping = [(i, v) for i, v in enumerate(project['metadata']['frame_mapping'][:50]) if v is not None]
print(f"   🗂️  Mapping échantillon (original→traité): {sample_mapping[:5]}...")

print(f"\n🎉 Pipeline SAM2 terminée avec succès!")


In [ ]:
project['objects']

In [ ]:
using_colab=True
if using_colab:
  from google.colab import drive
  import shutil

  # Monter le Drive
  drive.mount('/content/drive')

  # Créer directement le ZIP dans le Drive
  shutil.make_archive(f'/content/drive/MyDrive/{VIDEO_NAME}', 'zip', '/content/videos')

  print("Dossier vidéo sauvegardé en ZIP dans votre Google Drive !")

